In [1]:
from tqdm import tqdm
import pandas as pd
import cv2

from common.metrics import calculate_ssim, calculate_psnr, calculate_lpips, calculate_gmsd
from common.config import BLUR_PARAM_RANGES

from pathlib import Path

def run_metrics(
        dataset_path: Path,
        original_path: Path,
        deblurred_path: Path,
        save: bool = False
):
    df = pd.read_parquet(dataset_path)

    for blur_type in BLUR_PARAM_RANGES.keys():
        for key in tqdm(df.dropna()['key'], desc=f"Metrics: {blur_type:<8}\t"):
            img1_path = original_path / f"{key}.png"
            img2_path = deblurred_path / blur_type /f"{key}.png"

            img1 = cv2.imread(img1_path)
            img2 = cv2.imread(img2_path)

            ssim = calculate_ssim(img1, img2)
            psnr = calculate_psnr(img1, img2)
            lpips = calculate_lpips(img1, img2)
            gmsd = calculate_gmsd(img1, img2)

            df.loc[df['key'] == key, f'ssim_{blur_type}'] = ssim
            df.loc[df['key'] == key, f'psnr_{blur_type}'] = psnr
            df.loc[df['key'] == key, f'lpips_{blur_type}'] = lpips
            df.loc[df['key'] == key, f'gmsd_{blur_type}'] = gmsd
    
    if save:
        df.to_parquet(dataset_path)
    
    return df

In [2]:
from common.config import TEST_ORIGINAL_DIR, TEST_DEBLURRED_DIR, IDPA_DIR

name = "pseudoinverse_nokernel"

df = run_metrics(
    dataset_path=IDPA_DIR / f"{name}.parquet",
    original_path=TEST_ORIGINAL_DIR / "00000",
    deblurred_path=TEST_DEBLURRED_DIR / name
)

Metrics: box     	:   2%|▏         | 21/950 [00:42<31:20,  2.02s/it]


KeyboardInterrupt: 